In [1]:
#!pip install langchain openai

In [2]:
# Azure OpenAI
AZURE_OPENAI_ENDPOINT = "https://ai-new-agent-resource.openai.azure.com/"
AZURE_OPENAI_KEY = "3KZOVIIWVSSadEQyVLMr722pVoXVOl6BvatHtqrAsnmSV06FL1SCJQQJ99BIACHYHv6XJ3w3AAAAACOGGKR1"
DEPLOYMENT_NAME = "gpt-4o"

In [3]:
#!pip install langchain_community

In [5]:
from langchain.chat_models import AzureChatOpenAI

In [11]:
llm = AzureChatOpenAI(
    openai_api_base=AZURE_OPENAI_ENDPOINT,
    openai_api_key=AZURE_OPENAI_KEY,
    openai_api_version="2023-03-15-preview",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_type="azure",
)

/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://ai-new-agent-resource.openai.azure.com/ to https://ai-new-agent-resource.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:181: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:189: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.co

In [12]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(" You are a helpful assitant. \n User: { user_input}\n Assistant:")
print(prompt_template)

input_variables=[' user_input'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[' user_input'], input_types={}, partial_variables={}, template=' You are a helpful assitant. \n User: { user_input}\n Assistant:'), additional_kwargs={})]


In [13]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt_template)

In [14]:
while True:
  user_input = input("You: ")
  if user_input == "exit":
    break
  response = chain.run(user_input)
  print("Assistant: ", response)

You: Hi
Assistant:  Hello! 😊 How can I assist you today?
You: Tell ,me about delhi
Assistant:  Certainly! Delhi is a vibrant and historically rich city that serves as the capital of India. It is one of the oldest cities in the world, with a history that spans thousands of years. Here's an overview of the city:

### 1. **Geography and Administrative Setup**
   - Delhi is situated in northern India, bordered by Haryana on three sides and Uttar Pradesh to the east.
   - It is divided into two main parts:
     - **Old Delhi**: Known for its historic landmarks, bustling bazaars, and traditional culture.
     - **New Delhi**: Planned by British architect Edwin Lutyens in the early 20th century, it serves as the seat of the Indian government and features wide roads, formal gardens, and colonial-era buildings.
   - Administratively, Delhi is a Union Territory with special status and is governed by both the central government and its own legislative assembly.

### 2. **Historical Significance**

In [19]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the capital of France?")
]

print(messages)
Output = llm.invoke(messages)
print(Output)
print(Output.content)

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={})]
content='The capital of France is **Paris**.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 24, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'finish_reason': 'stop', 'logprobs': None} id='run--570d48be-f6b8-45ff-a60d-6fc21214c317-0'
The capital of France is **Paris**.


In [20]:
%%time
prompt = "Tell me a joke"
response = llm.invoke(prompt)
print(response)

content="Sure! Here's one for you:\n\nWhy don’t skeletons fight each other?\n\nBecause they don’t have the guts!" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 11, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'finish_reason': 'stop', 'logprobs': None} id='run--a6caa1f7-914f-44bf-a2b9-b977500cd5c6-0'
CPU times: user 13.3 ms, sys: 1.07 ms, total: 14.4 ms
Wall time: 519 ms


In [21]:
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())

In [22]:
%%time
prompt = "Tell me a joke"
response = llm.invoke(prompt)
print(response)

content="Sure! Here's a fun one for you:\n\nWhy don’t skeletons fight each other?\n\nBecause they don’t have the guts! 🦴😄" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 11, 'total_tokens': 43, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'finish_reason': 'stop', 'logprobs': None} id='run--63b3cd3f-2b4f-494a-a081-c812c878d650-0'
CPU times: user 18.5 ms, sys: 0 ns, total: 18.5 ms
Wall time: 602 ms


In [23]:
%%time
prompt = "Tell me a joke"
response = llm.invoke(prompt)
print(response)

content="Sure! Here's a fun one for you:\n\nWhy don’t skeletons fight each other?\n\nBecause they don’t have the guts! 🦴😄" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 11, 'total_tokens': 43, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'finish_reason': 'stop', 'logprobs': None} id='run--63b3cd3f-2b4f-494a-a081-c812c878d650-0' usage_metadata={'total_cost': 0}
CPU times: user 1.7 ms, sys: 0 ns, total: 1.7 ms
Wall time: 1.68 ms


In [24]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [25]:
%%time
prompt = "Tell me a joke"
response = llm.invoke(prompt)
print(response)

content="Sure! Here's one:\n\nWhy don’t skeletons fight each other?\n\nBecause they don’t have the guts! 😄" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 11, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'finish_reason': 'stop', 'logprobs': None} id='run--cd79b3fd-86d2-4c90-8214-9f9d8f557653-0'
CPU times: user 36 ms, sys: 139 µs, total: 36.2 ms
Wall time: 566 ms


In [26]:
%%time
prompt = "Tell me a joke"
response = llm.invoke(prompt)
print(response)

content="Sure! Here's one:\n\nWhy don’t skeletons fight each other?\n\nBecause they don’t have the guts! 😄" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 11, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'finish_reason': 'stop', 'logprobs': None} id='run--cd79b3fd-86d2-4c90-8214-9f9d8f557653-0' usage_metadata={'total_cost': 0}
CPU times: user 3.66 ms, sys: 0 ns, total: 3.66 ms
Wall time: 3.5 ms


In [27]:
%%time
prompt = 'write a rock song about moon.'
response = llm.invoke(prompt)
print(response.content)

**"Lunar Calling"**

*(Verse 1)*  
Under the vast black velvet sky,  
I see the moon, it’s my alibi.  
Silver beams cut through the night,  
Pulling me closer with its ancient light.  

Oh, its craters hold the secrets untold,  
Whispers of stories from ages old.  
Gravity's pull, it’s got me hooked,  
A cosmic dance, a page unwritten in the book.  

*(Chorus)*  
Moon, oh moon, you’re calling me home,  
A beacon shining where the wild ones roam.  
Through the dark, you light my way,  
Chasing shadows till the break of day.  

Moon, oh moon, in your quiet embrace,  
I’m lost in the rhythm of celestial space.  
Rising high like a tidal wave,  
You’re the rebel that the night can’t tame.  

*(Verse 2)*  
Born to wander, I’m a soul set free,  
But you’ve got a hold on gravity and me.  
Ocean tides bow to your command,  
You’re the queen of the sky, the ruler of this land.  

I feel your pull like a magnet’s force,  
Guiding my heart on a lunar course.  
A midnight rider with stars in my ey

In [28]:
%%time
prompt = 'write a rock song about moon.'
response = llm.invoke(prompt)
print(response.content)

**"Lunar Calling"**

*(Verse 1)*  
Under the vast black velvet sky,  
I see the moon, it’s my alibi.  
Silver beams cut through the night,  
Pulling me closer with its ancient light.  

Oh, its craters hold the secrets untold,  
Whispers of stories from ages old.  
Gravity's pull, it’s got me hooked,  
A cosmic dance, a page unwritten in the book.  

*(Chorus)*  
Moon, oh moon, you’re calling me home,  
A beacon shining where the wild ones roam.  
Through the dark, you light my way,  
Chasing shadows till the break of day.  

Moon, oh moon, in your quiet embrace,  
I’m lost in the rhythm of celestial space.  
Rising high like a tidal wave,  
You’re the rebel that the night can’t tame.  

*(Verse 2)*  
Born to wander, I’m a soul set free,  
But you’ve got a hold on gravity and me.  
Ocean tides bow to your command,  
You’re the queen of the sky, the ruler of this land.  

I feel your pull like a magnet’s force,  
Guiding my heart on a lunar course.  
A midnight rider with stars in my ey

In [29]:
%%time
prompt = 'write a rock song about moon and sun.'
response = llm.invoke(prompt)
print(response.content)

**"Dance of the Moon and Sun"**  
*(Rock Anthem)*  

**[Verse 1]**  
In the midnight sky, the moon takes its throne,  
Casting silver light on the world alone.  
She whispers secrets to the restless seas,  
Dreams of freedom carried on the breeze.  

The sun awakens, burning fierce and bright,  
Chasing shadows, banishing the night.  
With golden rays, it paints the earth alive,  
A blazing beacon where the wild hearts thrive.  

**[Pre-Chorus]**  
They’re opposites, but they’re bound as one,  
Locked in orbit, moon and sun.  
A cosmic waltz, a timeless race,  
Forever spinning in a fiery embrace.  

**[Chorus]**  
Oh, the moon and the sun collide,  
In the heavens where shadows and flames reside.  
They chase each other, round and round,  
In the endless sky, no limit, no ground.  
Dance of the moon and sun,  
Forever chasing, never done.  

**[Verse 2]**  
The moon’s a poet, quiet and wise,  
Her heart’s a mystery, hidden in her guise.  
The sun’s a renegade, wild and free,  
A roari

In [30]:
%%time
prompt = 'write a rock new song about moon and sun.'
for chunk in llm.stream(prompt):
  print(chunk.content, end="", flush=True)

**Title: "Moon and Sun"**

*(Verse 1)*  
In the still of the night, the moon takes the throne,  
Ruling the shadows, a queen all alone.  
Her silver light whispers secrets untold,  
A quiet rebel, fierce and bold.  

But the sun, oh the sun, he burns through the skies,  
Chasing the darkness, igniting our lives.  
A fiery king with a heart ablaze,  
They’re opposites dancing through endless days.  

*(Pre-Chorus)*  
They’re caught in a cosmic game,  
Forever circling, never the same.  
Two lovers who can’t collide,  
But still, they light the tide.  

*(Chorus)*  
Moon and Sun, chasing through the night,  
Lost in the pull of each other’s light.  
Bound by a force they can’t outrun,  
The eternal dance of the Moon and Sun.  

*(Verse 2)*  
When the moon fades away, the sun takes the stage,  
Painting the skies in a golden rage.  
But deep down inside, they’re longing to meet,  
Two kindred souls, incomplete.  

She watches him rise, he sees her descend,  
A love so vast it’ll never end